In [1]:
import json

import requests
from flask import Flask
from flask import request, render_template

from paper import index
from paper.utils import post_request

from paper.config import SEARCH_PATH, SEARCH_PORT
from paper.config import RANK_PATH, RANK_PORT, ANALYZER_PATH, ANALYZER_PORT


app = Flask(__name__)

@app.route("/")
def hello():
    """Render user interface with search form from templates"""
    return render_template("index.html")


@app.route(SEARCH_PATH, methods=['GET', 'POST'])
def search():
    """
    Search server provide UI hosting and all search workflow
    Receive ajax post request from user interface with meta and search  
    """
    if request.method=='POST':    
        data = request.json
        meta = data['meta']
        query = data['query']
        
        analyze_json = {'textes': [query]}
        r_analyzing_query = post_request(ANALYZER_PORT, ANALYZER_PATH, analyze_json)
        stemmed_query = r_analyzing_query['stemmed_docs'][0]
        query_lang = r_analyzing_query['langs'][0]

        query_json = {"stemmed_query": stemmed_query, "meta":"lalka"}  
        r_rank = post_request(RANK_PORT, RANK_PATH, query_json)
        ranked_ids = r_rank["ranked_ids"]
        
        ranked_docs = index.get_docs(ranked_ids)
        
        return json.dumps({'status': "ok", "results": ranked_docs})
    else:
        return "give me some text by post json {text: 'your text here'}"
    

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=SEARCH_PORT)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:13568/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Nov/2018 13:00:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:00:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2018 13:00:32] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:02:02] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:02:32] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:02:49] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:25:31] "POST /search HTTP/1.1" 200 -
[2018-11-27 13:26:06,323] ERROR in app: Exception on /search [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/site-packages/flask/app.py", line 1718, in handle_user_exception
    rera